In [5]:
import pandas as pd

In [6]:
infile = "catalysts-11-00286-s001.csv"

try:
    df = pd.read_csv(infile, encoding = 'utf-8')
except:
    df = pd.read_csv(infile, encoding = 'latin1')

In [8]:
df.to_excel('catalyst.xlsx', index=False)

In [7]:
print(df.columns)

Index(['Enzyme Name', 'UniprotID', 'ECNumber', 'Organism', 'PDB ID',
       'Chain ID', 'Mutated Site', 'Reaction', 'Reaction Substrate ',
       'SMILES of Substrate', 'Product', 'Temperature(¡ãC)', 'pH',
       'Catalytic Efficicnecy Change', 'PubMedID'],
      dtype='object')


In [9]:
print(list(df['PDB ID']))
print(list(df['Reaction Substrate ']))
print(list(df['SMILES of Substrate']))

all_smiles = zip(list(df['PDB ID']),list(df['Reaction Substrate ']), list(df['SMILES of Substrate']))
unique_smiles = set(all_smiles)

['1MJ5', '2D73', '2D73', '3IFA', '1SWW', '1SWW', '1SWW', '1SWW', '1SWW', '1SWW', '1A59', '1A59', '1A59', '1A59', '1A59', '3DYN', '3DYN', '3TG0', '3TG0', '3TG0', '3TG0', '3TG0', '3TG0', '3TG0', '3TG0', '1EYK', '1EYK', '1EYK', '1EYK', '1EYK', '1FBH', '1DP0', '1JYV', '1JYW', '1JYV', '1JYW', '1JYV', '1JYW', '1JYW', '1JYV', '1DP0', '2ST1', '2ST1', '1B12', '1B12', '1B12', '1B12', '1B12', '1B12', '1B12', '1B12', '1B12', '1B12', '1B12', '1B12', '2BLS', '1B9V', '1B9V', '1B9V', '1B9V', '1B9V', '1B9V', '1BC2', '1BC2', '1BC2', '1BC2', '1BC2', '1BC2', '1HNY', '1XX2', '1XX2', '1XX2', '1XX2', '1PNK', '1PNK', '1PNK', '1GM7', '1RLA', '1RLA', '1RLA', '1RLA', '1RLA', '1RLA', '1FTA', '1HZY', '1HZY', '1HZY', '1HZY', '1HZY', '1HZY', '1HZY', '1FUG', '1FUG', '1FUG', '1FUG', '2G2U', '2G2U', '2G2U', '2G2U', '1MAT', '1QJV', '1QJV', '1QJV', '1QJV', '1QJV', '1PHR', '1PHR', '1PHR', '1PHR', '1PHR', '1PHR', '2UXY', '2UXY', '2A9G', '2A9G', '2A9G', '2A9G', '2A9G', '1YTS', '1YTS', '1YTS', '1YTS', '1YTS', '1YTS', '1YTS',

In [10]:
outfile  = "substrates.txt"
#PDB_id = list(df['PDB ID'])
#PDB_id = set(PDB_id)
with open(outfile, 'w+') as f:
    for line in unique_smiles:
        line = [str(x) for x in line]
        f.write('\t'.join(line) + "\n")

In [14]:
outfile  = "SMILES.txt"
smiles = list(df['SMILES of Substrate'])
SMILES = set(smiles)
with open(outfile, 'w+') as f:
    for line in SMILES:
        f.write(str(line) + "\n")

In [16]:
import pybel

ModuleNotFoundError: No module named 'pybel'

In [4]:
infile = "pdb_ids.txt"
names = []

with open(infile, 'r') as f:
    for line in f:
        line = line.strip()
        names.append(line)
names = set(names)
with open('pdb_set.txt','a+') as f:
    for i in names:
        f.write(i + '\n')


In [13]:
import pubchempy as pcp

In [27]:
df = pd.read_csv('updated_substrates.txt', sep='\t', header=None)
df.columns = ['ID', 'Compound_Name', 'SMILES']

# Loop through each row to check for 'nan' in the SMILES column
for idx, row in df.iterrows():
    if pd.isna(row['SMILES']):
        compound_name = row['Compound_Name']
        try:
            # Fetch the SMILES using PubChemPy based on the compound name
            compound = pcp.get_compounds(compound_name, 'name')
            if compound:
                # Replace 'nan' with the fetched SMILES
                df.at[idx, 'SMILES'] = compound[0].canonical_smiles
        except Exception as e:
            print(f"Error fetching SMILES for {compound_name}: {e}")

# Save the updated DataFrame back to a file
df.to_csv('updated_substrates_2.txt', sep='\t', index=False)

Error fetching SMILES for D-Fructose 1,6-bisphosphate: Remote end closed connection without response


In [23]:
df = pd.read_csv('updated_substrates.txt', sep='\t', header=None)
df.columns = ['ID', 'Compound_Name', 'SMILES']

# Check for empty SMILES rows
empty_smiles_rows = df[df['SMILES'].isna() | (df['SMILES'] == '')]

# Display rows with empty SMILES
if not empty_smiles_rows.empty:
    print("Rows with empty SMILES found:")
    print(empty_smiles_rows)
else:
    print("No empty SMILES rows found.")

Rows with empty SMILES found:
      ID                Compound_Name SMILES
28  3IFA  D-Fructose 1,6-bisphosphate    NaN
63  1FTA  D-Fructose 1,6-bisphosphate    NaN
69  1QJV                       Pectin    NaN


In [31]:
compound = pcp.get_compounds('D-Fructose-1,6-bisphosphate', 'name')
if compound:
    print("SMILES:", compound[0].canonical_smiles)
else:
    print("Compound not found")
    

SMILES: C(C1C(C(C(O1)(COP(=O)(O)O)O)O)O)OP(=O)(O)O


In [34]:
import pandas as pd
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import AllChem

# Load the file into a DataFrame
df = pd.read_csv('substrates.txt', sep='\t', header=None)
df.columns = ['ID', 'Compound_Name', 'SMILES']

# Function to fetch SMILES and convert to cyclic form if possible
def fetch_cyclic_smiles(compound_name):
    try:
        # Fetch SMILES from PubChem
        compound = pcp.get_compounds(compound_name, 'name')
        if compound:
            smiles = compound[0].canonical_smiles
            # Convert to RDKit molecule
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                # Add explicit hydrogens to the molecule
                mol = Chem.AddHs(mol)
                # Attempt to generate the 3D conformer to ensure ring closure
                AllChem.EmbedMolecule(mol)
                # Use RDKit to convert to canonical SMILES in cyclic form if applicable
                cyclic_smiles = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
                return cyclic_smiles
    except Exception as e:
        print(f"Error processing {compound_name}: {e}")
    return None

# Loop through each row to check for NaN or empty SMILES
for idx, row in df.iterrows():
    if pd.isna(row['SMILES']) or row['SMILES'] == '':
        compound_name = row['Compound_Name']
        
        # Try to fetch SMILES in cyclic form
        cyclic_smiles = fetch_cyclic_smiles(compound_name)
        
        # If SMILES is found, update it in the DataFrame
        if cyclic_smiles:
            df.at[idx, 'SMILES'] = cyclic_smiles
        else:
            print(f"Cyclic SMILES not found for {compound_name}")

# Save the updated DataFrame back to a file
df.to_csv('updated_substrates_with_cyclic_smiles.txt', sep='\t', index=False)


Cyclic SMILES not found for D-Fructose 1,6-bisphosphate
Cyclic SMILES not found for D-Fructose 1,6-bisphosphate
Cyclic SMILES not found for Pectin


In [36]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms

# Load the file into a DataFrame
df = pd.read_csv('cyclic_smiles.txt', sep='\t', header=None)
df.columns = ['ID', 'Compound_Name', 'SMILES']

# Loop through each SMILES and create a PDB file
for idx, row in df.iterrows():
    compound_name = row['Compound_Name']
    smiles = row['SMILES']
    
    if pd.isna(smiles) or smiles == '':
        print(f"Skipping {compound_name} as it has no SMILES.")
        continue
    
    try:
        # Convert SMILES to a molecule
        mol = Chem.MolFromSmiles(smiles)
        
        # Add hydrogens
        mol = Chem.AddHs(mol)
        
        # Generate 3D coordinates
        AllChem.EmbedMolecule(mol, randomSeed=0xf00d)
        AllChem.UFFOptimizeMolecule(mol)  # Energy minimization
        
        # Save as PDB file
        pdb_filename = f"{compound_name.replace(' ', '_')}.pdb"
        with open(pdb_filename, 'w') as f:
            f.write(Chem.MolToPDBBlock(mol))
        
        print(f"PDB file for {compound_name} saved as {pdb_filename}")
    
    except Exception as e:
        print(f"Error processing {compound_name}: {e}")


[20:08:27] SMILES Parse Error: syntax error while parsing: SMILES
[20:08:27] SMILES Parse Error: Failed parsing SMILES 'SMILES' for input: 'SMILES'


Error processing Compound_Name: Python argument types in
    rdkit.Chem.rdmolops.AddHs(NoneType)
did not match C++ signature:
    AddHs(class RDKit::ROMol mol, bool explicitOnly=False, bool addCoords=False, class boost::python::api::object onlyOnAtoms=None, bool addResidueInfo=False)
PDB file for p-Nitrophenyl-beta-D-glucoside saved as p-Nitrophenyl-beta-D-glucoside.pdb
PDB file for 4-Nitrophenyl phosphate saved as 4-Nitrophenyl_phosphate.pdb
PDB file for Ampicillin saved as Ampicillin.pdb
PDB file for Ampicillin saved as Ampicillin.pdb
PDB file for Phosphotyrosine saved as Phosphotyrosine.pdb
PDB file for GDP-alpha-D-Glucose saved as GDP-alpha-D-Glucose.pdb
PDB file for Nitrocefin saved as Nitrocefin.pdb
PDB file for DIBOA glucoside saved as DIBOA_glucoside.pdb
PDB file for Benzylpenicillin saved as Benzylpenicillin.pdb
PDB file for S-Ethyl-L-cysteine saved as S-Ethyl-L-cysteine.pdb
PDB file for Peptide saved as Peptide.pdb
PDB file for 2-Aminobenzoyl-GGFLRKHGQ-N-(2,4-dinitrophenyl)et

[20:08:28] UFFTYPER: Unrecognized atom type: *_ (0)
[20:08:28] UFFTYPER: Unrecognized atom type: *_ (7)
[20:08:28] UFFTYPER: Unrecognized atom type: *_ (0)
[20:08:28] UFFTYPER: Unrecognized atom type: *_ (7)


PDB file for Penicillin G saved as Penicillin_G.pdb
PDB file for 4-Nitrophenyl acetate saved as 4-Nitrophenyl_acetate.pdb
PDB file for Triphosphate saved as Triphosphate.pdb
PDB file for Acetylthiocholine saved as Acetylthiocholine.pdb
PDB file for P1,P3-Bis(5'-adenosyl)-triphosphate saved as P1,P3-Bis(5'-adenosyl)-triphosphate.pdb
PDB file for Cefoxitin saved as Cefoxitin.pdb
PDB file for 4-Nitrophenyl-beta-D-galactopyranoside saved as 4-Nitrophenyl-beta-D-galactopyranoside.pdb
PDB file for Ceftazidime saved as Ceftazidime.pdb
PDB file for Ceftazidime saved as Ceftazidime.pdb
PDB file for 3,4-Dinitrophenyl-beta-D-galactopyranoside saved as 3,4-Dinitrophenyl-beta-D-galactopyranoside.pdb
PDB file for 2-Nitro-5-[(Phenylacetyl)amino]benzoic acid saved as 2-Nitro-5-[(Phenylacetyl)amino]benzoic_acid.pdb
PDB file for ATP saved as ATP.pdb
PDB file for p-Nitrophenyl-beta-D-fucopyranoside saved as p-Nitrophenyl-beta-D-fucopyranoside.pdb
PDB file for Phosphonoacetaldehyde saved as Phosphonoaceta

In [1]:
import pandas as pd

# Load the original Excel file
df = pd.read_excel('catalyst.xlsx')

# Extract unique rows based on Enzyme Name, PDB ID, Chain ID, and Mutated Site
unique_enzymes = df[['Enzyme Name', 'PDB ID', 'Chain ID', 'Mutated Site']].drop_duplicates()

# Function to parse the mutation details from 'Mutated Site' (e.g., "L177A")
def parse_mutation(mutation_str):
    original_residue = mutation_str[7]  # First character after "mutant "
    position = ''.join(filter(str.isdigit, mutation_str))  # Extract numeric position
    new_residue = mutation_str[-1]  # Last character for the new residue
    return original_residue, int(position), new_residue

# Apply parsing to extract original residue, position, and new residue
unique_enzymes[['Original Residue', 'Residue Position', 'New Residue']] = unique_enzymes['Mutated Site'].apply(parse_mutation).apply(pd.Series)

# Save the structured data to 'unique_enzymes.csv'
unique_enzymes.to_csv('unique_enzymes.csv', index=False)

print("unique_enzymes.csv has been generated successfully.")


unique_enzymes.csv has been generated successfully.


In [4]:
import pandas as pd
import pymol2
import os

# Path to your data folder containing the downloaded PDB files
data_folder = "../data"
output_folder = "../mutated_structures"
os.makedirs(output_folder, exist_ok=True)

# Load the unique enzyme data from the CSV file
unique_enzymes = pd.read_csv('unique_enzymes.csv')

# Define function to mutate each enzyme
def mutate_enzyme(pdb_filepath, chain, residue_position, new_residue, output_filename):
    with pymol2.PyMOL() as pymol:
        pymol.cmd.load(pdb_filepath)  # Load the PDB file from the local directory
        pymol.cmd.wizard("mutagenesis")
        pymol.cmd.get_wizard().set_mode(new_residue)  # Set the target amino acid for mutation
        pymol.cmd.get_wizard().do_select(f"resi {residue_position} and chain {chain}")
        pymol.cmd.get_wizard().apply()  # Apply the mutation
        pymol.cmd.set_wizard()  # Turn off the wizard
        pymol.cmd.save(output_filename)  # Save the mutated structure as PDB

# Loop through each unique enzyme and apply mutations
for idx, row in unique_enzymes.iterrows():
    pdb_id = row['PDB ID']
    chain = row['Chain ID']
    residue_position = row['Residue Position']
    new_residue = row['New Residue']
    enzyme_name = row['Enzyme Name'].replace(" ", "_")
    
    # Define file paths
    pdb_filepath = os.path.join(data_folder, f"{pdb_id}.pdb")
    output_filename = os.path.join(output_folder, f"mutated_{enzyme_name}_{pdb_id}_res{residue_position}_{new_residue}.pdb")
    
    # Check if the PDB file exists before mutating
    if not os.path.isfile(pdb_filepath):
        print(f"PDB file for {pdb_id} not found in the data folder. Skipping {enzyme_name}.")
        continue
    
    # Perform the mutation
    print(f"Processing {enzyme_name} ({pdb_id}) - Mutating residue {residue_position} to {new_residue}")
    mutate_enzyme(pdb_filepath, chain, residue_position, new_residue, output_filename)


ModuleNotFoundError: No module named 'pymol2'